# Natural Language Processing using Bidirectional-LSTM

### Loading Dependencies

In [1]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, LSTM
from keras.layers.wrappers import Bidirectional
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt 
%matplotlib inline

Using TensorFlow backend.


### Loading Data

In [2]:
n_unique_words = 10000 
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words)

### Working with Data

In [3]:
max_review_length = 100
pad_type = trunc_type = 'pre'
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

### Creating the model

In [4]:
model = Sequential()
model.add(Embedding(n_unique_words, 64, input_length=max_review_length)) 
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(256, dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           640000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 100, 64)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               657408    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 1,297,921
Trainable params: 1,297,921
Non-trainable params: 0
_________________________________________________________________


### Configure model

In [5]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Model Checkpoints

In [6]:
output_dir = 'model_output/Bidirectional-LSTM/'
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Training the model

In [7]:
model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_valid, y_valid), callbacks=[modelcheckpoint])

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 308s 12ms/step - loss: 0.4875 - acc: 0.7547 - val_loss: 0.3711 - val_acc: 0.8492
Epoch 2/5
25000/25000 [==============================] - 307s 12ms/step - loss: 0.2888 - acc: 0.8846 - val_loss: 0.3352 - val_acc: 0.8526
Epoch 3/5
25000/25000 [==============================] - 304s 12ms/step - loss: 0.2382 - acc: 0.9082 - val_loss: 0.3488 - val_acc: 0.8483
Epoch 4/5
25000/25000 [==============================] - 304s 12ms/step - loss: 0.2020 - acc: 0.9252 - val_loss: 0.3656 - val_acc: 0.8428
Epoch 5/5
25000/25000 [==============================] - 303s 12ms/step - loss: 0.1698 - acc: 0.9376 - val_loss: 0.4138 - val_acc: 0.8310


### Model Evaluation

In [ ]:
model.load_weights(output_dir+"weights.02.hdf5")
y_hat = model.predict_proba(x_valid)

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
"{:0.2f}".format(roc_auc_score(y_valid, y_hat)*100.0)